In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
import joblib

In [2]:
X = pd.read_csv("D:/tabular-playground-series-apr-2022/train.csv")
y = pd.read_csv("D:/tabular-playground-series-apr-2022/train_labels.csv")

In [3]:
X.head()

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.0,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.0,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.0,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.0,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.0,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359


In [4]:
y.head()

,sequence,state
0,0,0
1,1,1
2,2,1
3,3,1
4,4,1


In [5]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1558080 entries, 0 to 1558079
Data columns (total 16 columns):
 #   Column     Non-Null Count    Dtype  
---  ------     --------------    -----  
 0   sequence   1558080 non-null  int64  
 1   subject    1558080 non-null  int64  
 2   step       1558080 non-null  int64  
 3   sensor_00  1558080 non-null  float64
 4   sensor_01  1558080 non-null  float64
 5   sensor_02  1558080 non-null  float64
 6   sensor_03  1558080 non-null  float64
 7   sensor_04  1558080 non-null  float64
 8   sensor_05  1558080 non-null  float64
 9   sensor_06  1558080 non-null  float64
 10  sensor_07  1558080 non-null  float64
 11  sensor_08  1558080 non-null  float64
 12  sensor_09  1558080 non-null  float64
 13  sensor_10  1558080 non-null  float64
 14  sensor_11  1558080 non-null  float64
 15  sensor_12  1558080 non-null  float64
dtypes: float64(13), int64(3)
memory usage: 190.2 MB


null 값은 없으며  
step은 모두 0 ~ 59의 60초의 값이며  
subject는 각각의 사람(피험자)임  
이를 seneor0 ~ 12 값을 통하여 state를 예측하는 문제이다

In [6]:
X[X["subject"]==4]["sequence"].value_counts()

10       60
280      60
24465    60
24246    60
24131    60
23644    60
23174    60
22920    60
22421    60
22179    60
20273    60
19117    60
18410    60
16788    60
16080    60
14814    60
14130    60
10987    60
9300     60
9273     60
8972     60
7981     60
5883     60
5245     60
2114     60
24709    60
Name: sequence, dtype: int64

In [7]:
X[X["sequence"]==0]

,sequence,subject,step,sensor_00,sensor_01,sensor_02,sensor_03,sensor_04,sensor_05,sensor_06,sensor_07,sensor_08,sensor_09,sensor_10,sensor_11,sensor_12
0,0,47,0,-0.196291,0.112395,1.000000,0.329204,-1.004660,-0.131638,-0.127505,0.368702,-0.1,-0.963873,-0.985069,0.531893,4.751492
1,0,47,1,-0.447450,0.134454,1.000000,-0.658407,0.162495,0.340314,-0.209472,-0.867176,0.2,-0.301301,0.082733,-0.231481,0.454390
2,0,47,2,0.326893,-0.694328,1.000000,0.330088,0.473678,1.280479,-0.094718,0.535878,1.4,1.002168,0.449221,-0.586420,-4.736147
3,0,47,3,0.523184,0.751050,1.000000,0.976991,-0.563287,-0.720269,0.793260,0.951145,-0.3,-0.995665,-0.434290,1.344650,0.429241
4,0,47,4,0.272025,1.074580,1.000000,-0.136283,0.398579,0.044877,0.560109,-0.541985,-0.9,1.055636,0.812631,0.123457,-0.223359
5,0,47,5,-1.073416,-1.714286,1.000000,-1.492920,0.469736,0.713538,-1.792350,-1.082443,-0.5,0.314306,0.664594,-1.682099,-2.838448
6,0,47,6,0.350850,0.439076,1.000000,0.870796,-0.596964,0.923710,0.769581,0.666412,0.2,-0.723266,-1.691552,0.842593,1.849531
7,0,47,7,-0.304482,-0.379202,1.000000,-0.218584,-0.059983,-1.389678,-0.182149,-0.029771,0.1,-0.070809,-0.774024,-0.612140,3.300938
8,0,47,8,-0.216383,1.106092,1.000000,0.677876,-0.071718,-0.456993,0.296903,0.111450,0.6,-0.525289,-0.334069,1.751029,0.515772
9,0,47,9,0.303709,-1.744748,1.000000,-0.896460,-0.755221,-0.254301,-0.354281,-0.591603,0.0,0.151012,-0.268359,-2.033951,4.509378


In [8]:
X["subject"].value_counts()

437    11940
1      10500
635     9480
647     8640
87      7740
       ...  
486      420
519      420
265      360
73       300
472      120
Name: subject, Length: 672, dtype: int64

In [9]:
np.unique(X["subject"])

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [10]:
index = np.unique(X[X["subject"]==0]["sequence"])

In [11]:
for i in index:
    print(y[y["sequence"]==i]["state"])

207    0
Name: state, dtype: int64
1124    0
Name: state, dtype: int64
1174    1
Name: state, dtype: int64
5008    1
Name: state, dtype: int64
6119    0
Name: state, dtype: int64
7069    0
Name: state, dtype: int64
7366    1
Name: state, dtype: int64
7968    0
Name: state, dtype: int64
8043    0
Name: state, dtype: int64
8196    0
Name: state, dtype: int64
8755    0
Name: state, dtype: int64
11071    1
Name: state, dtype: int64
11324    0
Name: state, dtype: int64
12334    0
Name: state, dtype: int64
17802    0
Name: state, dtype: int64
19912    1
Name: state, dtype: int64
21923    0
Name: state, dtype: int64
23630    0
Name: state, dtype: int64
24629    0
Name: state, dtype: int64
25891    0
Name: state, dtype: int64


In [12]:
label = []
for i in range(len(X)):
    label.append(y[y["sequence"]==X["sequence"][i]]["state"])

In [13]:
joblib.dump(label,"label.pkl")

['label.pkl']

In [14]:
len(label)

1558080

In [15]:
len(X)

1558080

In [16]:
label = np.array(label)

In [37]:
del X["sequence"], X["subject"]

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, label, test_size=0.2, shuffle=True)

In [18]:
base_tree_model = ExtraTreesClassifier()
base_tree_model.fit(X_train, y_train)

C:\Users\hjhhi\AppData\Local\Temp\ipykernel_6108\4284581087.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  base_tree_model.fit(X_train, y_train)


ExtraTreesClassifier()

In [20]:
base_tree_model.score(X_test, y_test)

0.9559393612651469

In [38]:
base_tree_model = ExtraTreesClassifier()
base_tree_model.fit(X, label)

C:\Users\hjhhi\AppData\Local\Temp\ipykernel_6108\4251204770.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  base_tree_model.fit(X, label)


ExtraTreesClassifier()

In [23]:
X_test = pd.read_csv("D:/tabular-playground-series-apr-2022/test.csv")

In [ ]:
del X_test["sequence"], X_test["subject"]

In [42]:
result = base_tree_model.predict(X_test)

In [81]:
submission = pd.read_csv("D:/tabular-playground-series-apr-2022/sample_submission.csv")

In [44]:
pd.Series(result).value_counts()

1    382651
0    350429
dtype: int64

In [45]:
X.columns

Index(['step', 'sensor_00', 'sensor_01', 'sensor_02', 'sensor_03', 'sensor_04',
       'sensor_05', 'sensor_06', 'sensor_07', 'sensor_08', 'sensor_09',
       'sensor_10', 'sensor_11', 'sensor_12'],
      dtype='object')

In [46]:
base_tree_model.feature_importances_

array([0.0674456 , 0.0710135 , 0.07191605, 0.0477834 , 0.07107647,
       0.07775715, 0.07281752, 0.07096971, 0.07115138, 0.07232761,
       0.07291174, 0.07671702, 0.0718343 , 0.08427854])

In [ ]:
submission["state"]

In [47]:
result

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [63]:
result_row = result.reshape(12218, -1)

In [82]:
for i in range(len(result_row)):
    result_row[i] = round(result_row[i].mean())

In [83]:
submission["state"] = result_row

In [84]:
submission

,sequence,state
0,25968,1
1,25969,1
2,25970,0
3,25971,0
4,25972,1
...,...,...
12213,38181,1
12214,38182,1
12215,38183,0
12216,38184,0


In [85]:
submission.to_csv("base_tree_submission.csv")